# 102 Spark optimizations

The goal of this lab is to understand some of the optimization mechanisms of Spark.

- Scala
    - [Spark programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)
    - [RDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/RDD.html)
    - [PairRDD APIs](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/rdd/PairRDDFunctions.html)
- Python
    - [Spark programming guide](https://spark.apache.org/docs/3.5.0/rdd-programming-guide.html)
    - [All RDD APIs](https://spark.apache.org/docs/3.5.0/api/python/reference/api/pyspark.RDD.html)

Use `Tab` for autocompletion, `Shift+Tab` for documentation.

In [ ]:
from pyspark.sql import SparkSession

# .master("local[N]") <-- ask for N cores on the driver
spark = SparkSession.builder \
.master("local[4]") \
.appName("Local Spark") \
.config('spark.ui.port', '4040') \
  .getOrCreate()
sc = spark.sparkContext

sc

## The weather dataset

Download the following ZIP files and unzip them inside the "datasets/big" folder of this repo (which is not committed).
- [weather-sample1.txt](https://big.csr.unibo.it/downloads/bigdata/weather-datasets-s1.zip) <-- start from this!
- [weather-sample10.txt](https://big.csr.unibo.it/downloads/bigdata/weather-datasets-s10.zip)
- [weather-full.txt](https://big.csr.unibo.it/downloads/bigdata/weather-datasets-full.zip)
  
The weather datasets are textual files with weather data from all over the world in year 2000 (collected from the [National Climatic Data Center](ftp://ftp.ncdc.noaa.gov/pub/data/noaa/) of the USA. The full one weighs 13GB, the other are samples of 10% (1.3GB) and 1% (130MB) respectively.
  - Sample row: 005733213099999**19580101**03004+51317+028783FM-12+017199999V0203201N00721004501CN0100001N9 **-0021**1-01391102681
  - The date in YYYYMMDD format is located at 0-based position 15-23
  - The temperatue in x10 Celsius degrees is located at 0-based positions 87-92

In the dataset folder you also have *weather-stations.csv*; it is a structured file with the description of weather stations collecting the weather data.

In [ ]:
# WEATHER structure: (usaf,wban,year,month,day,airTemperature,airTemperatureQuality)
def parseWeather(row):
    usaf = row[4:10]
    wban = row[10:15]
    year = row[15:19]
    month = row[19:21]
    day = row[21:23]
    airTemperature = row[87:92]
    airTemperatureQuality = row[92]

    return (usaf,wban,year,month,day,int(airTemperature)/10,airTemperatureQuality == '1')

# STATION structure: (usaf,wban,city,country,state,latitude,longitude,elevation,date_begin,date_end) 
def parseStation(row):
    def getDouble(str):
        return 0 if len(str)==0 else float(str)
    
    columns = [ x.replace("\"","") for x in row.split(",") ]
    latitude = getDouble(columns[6])
    longitude = getDouble(columns[7])
    elevation = getDouble(columns[8])
    return (columns[0],columns[1],columns[2],columns[3],columns[4],latitude,longitude,elevation,columns[9],columns[10])  

In [ ]:
rddWeather = sc.\
  textFile("../../../../datasets/big/weather-sample1.txt").\
  map(lambda x: parseWeather(x))
rddStation = sc.\
  textFile("../../../../datasets/weather-stations.csv").\
  map(lambda x: parseStation(x))

## 102-1 Simple job optimization

Optimize the two jobs (avg temperature and max temperature) by avoiding the repetition of the same computations and by enforcing a partitioning criteria.
- There are multiple methods to repartition an RDD: check the ```coalesce```, ```partitionBy```, and ```repartition``` methods on the documentation and choose the best one.
- Verify your persisted data in the web UI
- Verify the execution plan of your RDDs with ```rdd.toDebugString``` (shell only) or on the web UI

In [ ]:
# Average temperature for every month
rddWeather.\
  filter(lambda x: x[5]<999).\
  map(lambda x: (x[3], (x[5],1))).\
  reduceByKey(lambda v1, v2: (v1[0]+v2[0], v1[1]+v2[1])).\
  mapValues(lambda v: round(v[0]/v[1],2)).\
  collect()

In [ ]:
# Maximum temperature for every month
rddWeather.\
  filter(lambda x: x[5]<999).\
  map(lambda x: (x[3], (x[5],1))).\
  reduceByKey(lambda x, y: y if x<y else x).\
  collect()

### Solution

Caching:
- It is good to cache the RDD **after** the common transformations (including the partitioning) have been carried out
- Then, remember to reference the cached RDD
- Check cached RDD in the Spark UI

Partitioning:
- ```coalesce``` aggregates partitions without shuffling; ```repartition``` and ```partitionBy``` force a shuffle
- Differently from ```partitionBy```, ```coalesce``` and ```repartition``` do not associate a partitioning criteria to the RDD; thus, both jobs must reshuffle the cached data
- After the aggregation, data can be reduced to a single partition (only 12 rows, one per month)

In [ ]:
cachedRdd = rddWeather.\
  filter(lambda x: x[5]<999).\
  map(lambda x: (x[3], (x[5],1))).\
  partitionBy(8).\
  cache()

In [ ]:
# Average temperature for every month
cachedRdd.\
  reduceByKey(lambda v1, v2: (v1[0]+v2[0], v1[1]+v2[1]),1).\
  mapValues(lambda v: round(v[0]/v[1],2)).\
  collect()

In [ ]:
# Maximum temperature for every month
cachedRdd.\
  reduceByKey(lambda x, y: min(x,y),1).\
  collect()

## 102-2 RDD preparation

Check the five possibilities to prepare the Station RDD for subsequent processing and identify the best one.

In [ ]:
num_partitions = 8

# [0] and [1] are the fields composing the key; [3] and [7] are country and elevation, respectively
rddS1 = rddStation.\
  keyBy(lambda x: x[0] + x[1]).\
  partitionBy(num_partitions).\
  cache().\
  map(lambda kv: (kv[0],(kv[1][3],kv[1][7])))
rddS2 = rddStation.\
  keyBy(lambda x: x[0] + x[1]).\
  map(lambda kv: (kv[0],(kv[1][3],kv[1][7]))).\
  cache().\
  partitionBy(num_partitions)
rddS3 = rddStation.\
  keyBy(lambda x: x[0] + x[1]).\
  partitionBy(num_partitions).\
  map(lambda kv: (kv[0],(kv[1][3],kv[1][7]))).\
  cache()
rddS4 = rddStation.\
  keyBy(lambda x: x[0] + x[1]).\
  map(lambda kv: (kv[0],(kv[1][3],kv[1][7]))).\
  partitionBy(num_partitions).\
  cache()
rddS5 = rddStation.\
  map(lambda x: (x[0] + x[1], (x[3],x[7]))).\
  partitionBy(num_partitions).\
  cache()

### Solution

- keyBy() and map() break the partitioning, thus they must be issued before partitionBy()
- Anything that happens after cache() is not saved and must be recomputed each time;
  thus, it is good practice to cache() as later as possible
- rddS4 and rddS5 are the best options (the latter being less verbose)

In [ ]:
# Use this to check the partitioner associated with an RDD
print(rddS1.partitioner)

## 102-3 Joining RDDs

Define the join between rddWeather and rddStation and compute:
- The maximum temperature for every city
- The maximum temperature for every city in the UK: 
  - ```StationData.country == "UK"```
- Sort the results by descending temperature
  - ```map(lambda kv: (kv[1],kv[0]))``` to invert key with value and vice versa

Hints & considerations:
- Keep only temperature values <999
- Join syntax: ```rdd1.join(rdd2)```
  - Both RDDs should be structured as key-value RDDs with the same key: usaf + wban
- Consider partitioning and caching to optimize the join
  - [Scala only] Careful: it is not enough for the two RDDs to have the same number of partitions; they must have the same partitioner! To create a partitioning function, you must ```import org.apache.spark.HashPartitioner``` and then define ```p = new HashPartitioner(n)``` where ```n``` is the number of partitions to create.
- Verify the execution plan of the join in the web UI

### Solution

In [ ]:
# Clear the cache
for (id, rdd) in sc._jsc.getPersistentRDDs().items():         
    rdd.unpersist()

In [ ]:
# First version: caching the join
num_partitions = 8

rddS = rddStation.\
  map(lambda s: (s[0]+s[1], (s[2], s[3]))).\
rddW = rddWeather.\
  filter(lambda w: w[6]).\
  map(lambda w: (w[0]+w[1], w[5]))

rddJoin = rddW.join(rddS,num_partitions).cache()

rddJoin.\
  map(lambda j: (j[1][1][0], j[1][0])).\
  reduceByKey(lambda x,y: min(x,y)).\
  collect()

In [ ]:
rddJoin.\
  filter(lambda j: j[1][1][1] == "UK").\
  map(lambda j: (j[1][1][0], j[1][0])).\
  reduceByKey(lambda x,y: min(x,y)).\
  collect()

In [ ]:
rddJoin.\
  filter(lambda j: j[1][1][1] == "UK").\
  map(lambda j: (j[1][1][0], j[1][0])).\
  reduceByKey(lambda x,y: min(x,y)).\
  map(lambda kv: (kv[1],kv[0])).\
  sortByKey(False).\
  collect()

Comments:
- The first job is the one taking longer because it has to read everything from disk; the other two will be faster because they rely on the cached results
- In PySpark, the rddJoin may be internally cached by Spark even if the method is not explicitly called; since this behavior is not predictable, always declare caching when needed
- The sortByKey is implemented by Spark as an action (even though, theoretically, it is just a transformation); this leads to a fragmentation of the third job, that (in the UI) gets split into multiple jobs
- You will notice that Spark subdivides the input into splits of 32MB; in the WebUI you will see that:
    - In the first stage, partitions are quite uniform (no skew in the data)
    - In the second stage (after the shuffle for the join), the data remains well distributed, but with only few records in each partition. Thus, it could make sense to reduce the number of partitions during the join (to avoid having too many tasks), even though the time reduction is not significant (in this case)
- Further optimization can be achieved by caching the result of the aggregation, which is used by all the three jobs. Notice that, to be able to filter by country, the reduceByKey must include the country in the key; on the one hand, this means that the aggregation costs more (because instead of aggregating <city,temp> pairs, we aggregate <(city,country),temp> pairs); however, this means that the aggregation is done only once (instead og thrice), so this is ultimately much convenient.

In [ ]:
# Second version: caching the aggregation
num_partitions = 8

rddS = rddStation.\
  map(lambda s: (s[0]+s[1], (s[2], s[3]))).\
  partitionBy(num_partitions)
rddW = rddWeather.\
  filter(lambda w: w[6]).\
  map(lambda w: (w[0]+w[1], w[5])).\
  partitionBy(num_partitions)

rddAgg = rddW.\
  join(rddS).\
  map(lambda j: ((j[1][1][0],j[1][1][1]), j[1][0])).\
  reduceByKey(lambda x,y: min(x,y)).\
  cache()

rddAgg.collect()

In [ ]:
rddAgg.\
  filter(lambda a: a[0][1]=="UK").\
  collect()

In [ ]:
rddAgg.\
  filter(lambda a: a[0][1]=="UK").\
  map(lambda kv: (kv[1],kv[0])).\
  sortByKey(False).\
  collect()

## 102-4 Memory occupation

Use Spark's web UI to verify the space occupied by the provided RDDs.

*Warning*: in PySpark, StoraleLevels use serialization by default (see [documentation](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.StorageLevel.html)).

In [ ]:
from pyspark import StorageLevel

# Clear the cache
for (id, rdd) in sc._jsc.getPersistentRDDs().items():         
    rdd.unpersist()

memSerRdd = rddWeather.cache()
memRdd = memSerRdd.map(lambda x: x).persist(StorageLevel.MEMORY_AND_DISK_DESER)
diskRdd = memSerRdd.map(lambda x: x).persist(StorageLevel.DISK_ONLY)

### Solution

- Collecting/Saving to file is required to trigger the (lazy) evaluation
- memSerRdd's size will be less than half of memRdd's
- diskRdd's size will be approximately the same as memSerRdd (output to disk is always serialized)
- StorageLevel differ in Scala and Python:
    - In Scala, `cache()` is deserialized and you have `*_SER` StorageLevels for the serialized versions
    - In PySpark, `cache()` is serialized and you have `*_DESER` StorageLevels for the deserialized versions

In [ ]:
memSerRdd.collect()
memRdd.collect()
diskRdd.collect()

## 102-5 Evaluating different join methods

Consider the following scenario:
- We have a disposable RDD of Weather data (i.e., it is used only once): ```rddW```
- And we have an RDD of Station data that is used many times: ```rddS```
- Both RDDs are cached (```collect()```is called to enforce caching)

We want to join the two RDDS. Which option is best?
- Simply join the two RDDs
- Enforce on ```rddW1``` the same partitioner of ```rddS``` (and then join)
- Exploit broadcast variables

In [ ]:
num_partitions = 8

rddW = rddWeather.\
  filter(lambda w: w[5]<999).\
  keyBy(lambda w: w[0]+w[1]).\
  cache()

rddS = rddStation.\
  keyBy(lambda s: s[0]+s[1]).\
  partitionBy(num_partitions).\
  cache()

# Collect to enforce caching
rddW.collect()
rddS.collect()

In [ ]:
# Is it better to simply join the two RDDs..
rddX = rddW.\
  join(rddS).\
  map(lambda kv: (kv[1][1][2],kv[1][0][5])).\
  reduceByKey(lambda x,y: min(x,y),1)
print(rddX.toDebugString().decode("unicode_escape"))

In [ ]:
rddX.collect()

In [ ]:
# ..to enforce on rddW1 the same partitioner of rddS..
rddX = rddW.\
  partitionBy(num_partitions).\
  join(rddS).\
  map(lambda kv: (kv[1][1][2],kv[1][0][5])).\
  reduceByKey(lambda x,y: min(x,y),1)
print(rddX.toDebugString().decode("unicode_escape"))

In [ ]:
rddX.collect()

In [ ]:
# ..or to exploit broadcast variables?
bRddS = sc.broadcast(rddS.map(lambda s: (s[0], s[1][2])).collectAsMap())
rddJ = rddW.\
  map(lambda kv: (bRddS.value.get(kv[0]), kv[1][5])).\
  filter(lambda x: x[0] is not None)

rddX = rddJ.\
  reduceByKey(lambda x,y: min(x,y),1)
print(rddX.toDebugString().decode("unicode_escape"))

In [ ]:
rddX.collect()

### Solution

- Enforcing a partitioner doesn't make sense if the rdd is used in one shuffle operation only: the partitioning will be done directly during that shuffle
- Broadcasting can be very powerful to optimize joins, BUT only if the dataset is small enough to easily fit the memory of the executors; if the dataset is bigger, it should be an RDD